# Lab 5: 2x3 Slider Puzzle Solver

## Introduction
This notebook focuses on solving a 2x3 Slider Puzzle using classical search algorithms in Artificial Intelligence (AI).
The problem is modeled as a state-space search problem where each move of a tile results in a new state. The puzzle is to be solved by a non-autonomous utility-based agent, which seeks the least-cost path from an initial to a goal configuration.

You will see *uninformed search* algorithms like `breadth_first_search`, `depth_first_search` and `uniform_cost_search`, as well as *informed search* algorithms like `greedy_search` and `astar_search` in action. 


---

### Essential Libraries

In [16]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import queue
from copy import deepcopy
sns.set()

### Problem Representation
Each state of the 2x3 puzzle is represented as a 2D NumPy array.
The blank tile is represented by 0. The objective is to rearrange the tiles so that they appear in ascending order from 1 to 5, with the 0 tile in the bottom-right corner.

### Helper Functions

##### **print_state(state, moved_tile=None)**
**Purpose**: Nicely prints the current puzzle grid (2x3), with optional highlighting of the tile that was moved.

**Key Features**:
- Displays the board with clear borders.
- Highlights the last moved tile with a *tile* format to help track movement.

---

##### **get_neighbor(state)**
**Purpose**: Generates all valid next states of the puzzle by sliding a tile into the empty space (0).

**How it works**:
- Locates the blank (0) tile.
- Checks if it can move up, down, left, or right.
- Returns a list of tuples:(action, resulting_state_after_move)

---

##### **move_tile(state, direction)**
**Purpose**: Returns a new state by sliding the blank tile in a specified direction.

---

##### **state_to_tuple(state)**
**Purpose**: Converts the 2D NumPy array state into a hashable tuple, so it can be:
- Stored in dictionaries (came_from, cost_so_far)
- Used in sets and priority queues

**Why**: NumPy arrays can't be used as dictionary keys due to being mutable/unhashable.

---

##### **Summary of Movement Directions**

| Direction | Description                            | Visual Example (relative to blank) |
|-----------|----------------------------------------|-------------------------------------|
| `up`      | Moves the blank space **upward**       | blank swaps with tile above      |
| `down`    | Moves the blank space **downward**     | blank swaps with tile below      |
| `left`    | Moves the blank space **leftward**     | blank swaps with tile to the left|
| `right`   | Moves the blank space **rightward**    | blank swaps with tile to the right|


In [17]:
def print_state(state, title="", moved_tile=None):
    """
    Print the 2x3 puzzle state in a pretty grid with optional tile highlighting.
    """
    if title:
        print(f"\n{title}")
    
    print("+" + "------" * 3 + "+")  # top border
    for i in range(2):
        row = "|"
        for j in range(3):
            val = state[i, j]
            if val == 0:
                cell = "    "  # empty cell
            elif moved_tile is not None and val == moved_tile:
                cell = f">>{val:2d}<<"  # highlight moved tile
            else:
                cell = f" {val:2d}  "
            row += f"{cell}|"
        print(row)
        print("+" + "------" * 3 + "+")  # row separator

def state_to_tuple(state):
    return tuple(state.flatten())

def get_neighbors(state):
    neighbors = []
    x, y = np.argwhere(state == 0)[0]
    
    moves = {
        "up":    (x - 1, y),
        "down":  (x + 1, y),
        "left":  (x, y - 1),
        "right": (x, y + 1)
    }
    
    for action, (nx, ny) in moves.items():
        if 0 <= nx < 2 and 0 <= ny < 3:
            new_state = state.copy()
            new_state[x, y], new_state[nx, ny] = new_state[nx, ny], new_state[x, y]
            neighbors.append((action, new_state))
    
    return neighbors

In [18]:
INITIAL_STATE = np.array([
    [1, 0, 3],
    [4, 2, 5]
])

GOAL_STATE = np.array([
    [1, 2, 3],
    [4, 5, 0]
])

print_state(INITIAL_STATE, "Initial State: ")
print_state(GOAL_STATE, "Goal State: ")


Initial State: 
+------------------+
|  1  |    |  3  |
+------------------+
|  4  |  2  |  5  |
+------------------+

Goal State: 
+------------------+
|  1  |  2  |  3  |
+------------------+
|  4  |  5  |    |
+------------------+


### Heuristic Function with Manhatten Distance - For A*Star

In informed search algorithms like A*, a heuristic function estimates the cost from the current state to the goal. One common and effective heuristic for grid-based problems is the Manhattan Distance.

The Manhattan Distance is the total number of horizontal and vertical moves required to reach from one point to another in a grid.

For two points **A(x₁, y₁) and B(x₂, y₂) on a 2D grid**, the Manhattan Distance is calculated as:

**ManhattanDistance(A, B) = |x1-x2| + |y1-y2|**

#### In the context of a 2x3 sliding tile puzzle:

- Each tile has a goal position (where it should end up).
- The Manhattan Distance is calculated for each tile between its current position and its goal position.
- The heuristic value h(n) is the sum of all individual tile distances (excluding the blank tile).

This provides a good estimate of how far the current puzzle state is from the solution.

---

##### heuristic(state, goal)
**Purpose**: Calculates the Manhattan Distance heuristic between the current state and the goal.

**How it works**:
- For each tile in the grid (except 0), find its:
- Current position (x1, y1)
- Goal position (x2, y2)
- Compute the sum of abs(x1 - x2) + abs(y1 - y2) for all tiles.

In [19]:
def heuristic(state, goal):
    distance = 0
    for val in range(1, 5):
        x1, y1 = np.where(state == val)
        x2, y2 = np.where(goal == val)
        distance += abs(x1[0] - x2[0]) + abs(y1[0] - y2[0])
    return distance

### A*Star Search Algorithm

##### **Logic**
1. Initialize the **priority queue** (frontier) with the start node.
2. Keep track of:
   - `came_from`: which node led to the current node
   - `cost_so_far`: total cost from start to each node
3. While the frontier is not empty:
   - Pop the node with the **lowest f(n)**
   - If it's the goal → done
   - For each valid neighbor:
     - Calculate `new_cost = cost_so_far[current] + 1`
     - If neighbor is unexplored or has a lower cost:
       - Update `cost_so_far`
       - Add to frontier with `priority = new_cost + heuristic(neighbor, goal)`
       - Record the parent in `came_from`

---

##### **Output**
- `came_from`: A dictionary mapping each node to the `(parent_state, move)`
- `final_state`: The goal state reached
- `steps`: Total number of nodes expanded

---

##### **Reconstruct Path**
Once the goal state is reached, we **reconstruct the full path** by backtracking from the goal to the start using the `came_from` dictionary.

##### **Function**: `reconstruct_path(came_from, start, goal)`
**Steps**:
1. Start at the `goal` node.
2. Look up its parent in `came_from` until reaching the `start`.
3. Collect each `(move, state)` into a list.
4. Reverse the list to get the path from start → goal.

---

### Using A* for the 2x3 Slider Puzzle
- The search space is large: **6** possible states, though not all are reachable.
- We need to explore efficiently without checking all possibilities.
- A* gives **optimal solutions** using **heuristics** to guide the search.

---

In [20]:
def astar_search(initial_state, goal_state):
    frontier = queue.PriorityQueue()
    start_tuple = state_to_tuple(initial_state)
    goal_tuple = state_to_tuple(goal_state)
    
    frontier.put((0, start_tuple))
    
    came_from = {start_tuple: (None, None)}
    cost_so_far = {start_tuple: 0}
    state_map = {start_tuple: initial_state.copy()}
    
    while not frontier.empty():
        _, current = frontier.get()
        current_state = state_map[current]
        
        if current == goal_tuple:
            return came_from, goal_tuple, len(came_from)
        
        for action, neighbor in get_neighbors(current_state):
            neighbor_tuple = state_to_tuple(neighbor)
            new_cost = cost_so_far[current] + 1

            if neighbor_tuple not in cost_so_far or new_cost < cost_so_far[neighbor_tuple]:
                cost_so_far[neighbor_tuple] = new_cost
                priority = new_cost + heuristic(neighbor, goal_state)
                frontier.put((priority, neighbor_tuple))
                came_from[neighbor_tuple] = (current, action)
                state_map[neighbor_tuple] = neighbor.copy()
    
    return came_from, None, len(came_from)

def reverse_direction(direction):
    mapping = {
        "up": "DOWN",
        "down": "UP",
        "left": "RIGHT",
        "right": "LEFT"
    }
    return mapping[direction]


In [21]:
def reconstruct_path(came_from, start_tuple, goal_tuple):
    current = goal_tuple
    path = []

    while came_from[current][0] is not None:
        prev, action = came_from[current]
        path.append((action, current))
        current = prev

    path.reverse()
    return path


In [22]:
came_from, final_state, steps = astar_search(INITIAL_STATE, GOAL_STATE)

if final_state is not None:
    path = reconstruct_path(came_from, state_to_tuple(INITIAL_STATE), final_state)

    current_state = INITIAL_STATE.copy()
    print("Initial State:")
    print_state(current_state)

    for i, (action, flat_state) in enumerate(path):
        new_state = np.array(flat_state).reshape((2, 3))

        blank_pos_old = tuple(zip(*np.where(current_state == 0)))[0]
        blank_pos_new = tuple(zip(*np.where(new_state == 0)))[0]
        moved_tile = current_state[blank_pos_new]

        direction = reverse_direction(action)
        print(f"\nStep {i+1}: Move {direction} (tile {moved_tile})")
        print_state(new_state, moved_tile=moved_tile)

        current_state = new_state

    print(f"Puzzle solved in {len(path)} steps, {steps} nodes processed.")
else:
    print("No solution found.")


Initial State:
+------------------+
|  1  |    |  3  |
+------------------+
|  4  |  2  |  5  |
+------------------+

Step 1: Move UP (tile 2)
+------------------+
|  1  |>> 2<<|  3  |
+------------------+
|  4  |    |  5  |
+------------------+

Step 2: Move LEFT (tile 5)
+------------------+
|  1  |  2  |  3  |
+------------------+
|  4  |>> 5<<|    |
+------------------+
Puzzle solved in 2 steps, 6 nodes processed.
